# Progetto Foundations Of Computer Science
- *Filippo Algeri* - mat. 828825
- *Lucrezia Maggi* - mat. 837329
- *Marco Donzella* - mat. 829358


#### You have to work on the files:

1. [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
2. [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
3. [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
4. [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
5. [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes
1. It is mandatory to use GitHub for developing the project.
2. The project must be a jupyter notebook.
3. There is no restriction on the libraries that can be used, nor on the Python version.
4. To read those files, you need to use the encoding = 'latin-1' option.
5. All questions on the project must be asked in a public channel on [Zulip](https://focs.zulipchat.com/), otherwise no answer will be given.

Vengono importate le librerie necessarie per le analisi:

In [ ]:
import pandas as pd
import numpy as np

Si importano i datasets: `GoodBooks` e `GoodBooks-Ratings` non sembrano avere problemi di codifica. Gli altri tre dataset (`Book`, `bookRatings` e `users`) hanno numerosi problemi di codifica, che sarebbe opportuno correggere.

Per i dataset *Book*, *Book-Ratings*, *Users* è stata utilizzata una codifica 'latin-1', mentre per i dataset *GoodBooks* e *GoodBooks-Ratings* la codifica classica, 'utf-8' (avendo visto che era più adatta).

In [ ]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression='gzip', encoding='latin-1', delimiter=';')
books.head() 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression='gzip', encoding='latin-1', delimiter=';')
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression='gzip', encoding='latin-1', delimiter=';')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression='gzip', parse_dates= ['original_publication_year'], delimiter=',') # qua il delimiter cambia, non è più ';' ma ','
goodbooks.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [ ]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression='gzip', delimiter=',') # anche qui il delimiter è la virgola
goodbooks_ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [ ]:
print('Ecco le dimensioni iniziali dei dataset:')
print('books --> '+ str(books.shape))
print('book_ratings --> '+ str(book_ratings.shape))
print('goodbooks --> '+ str(goodbooks.shape))
print('goodbooks_ratings --> '+ str(goodbooks_ratings.shape))
print('users --> '+ str(users.shape))

Ecco le dimensioni iniziali dei dataset:
books --> (271359, 8)
book_ratings --> (1149780, 3)
goodbooks --> (99, 23)
goodbooks_ratings --> (99, 3)
users --> (278858, 3)


# 0. Preprocessing - Encoding problems
Presentando i vari dataset dei caratteri particolari ed incomprensibili (dovuti alla codifica), si è deciso di iniziare le analisi con una rapida fase di preprocessing, così da eliminare i pochi record illeggibili e di difficile comprensione.

#### Books

Viene di seguito riportata la percentuale di record che presentano problemi di codifica:

In [ ]:
len(books[books['Book-Title'].str.contains(r"[§Ô½©ÃÂ£¶¤¼@¹«îûúöð±¾]")])/len(books) # percentuale di record da scartare

0.024550503207927506

In [ ]:
books = books[books['Book-Title'].str.contains(r"[§Ô½©ÃÂ£¶¤¼@¹«îûúöð±¾]") == False]

#### Books-Ratings

Alcuni ISBN presentano uno spazio prima dell'ISBN reale; conviene eliminarlo per evitare problemi in analisi future.

In [ ]:
book_ratings['ISBN'] = book_ratings['ISBN'].str.strip()

Sono stati rimossi i caratteri più strani all'interno dell'ISBN, ad eccezione dello '-', che è stato semplicemente rimosso all'interno dei valori (in quanto l'ISBN sono caratterizzati da tale carattere, che, però, solitamente non viene riportato).

In [ ]:
book_ratings = book_ratings[book_ratings['ISBN'].str.contains(r"[§Ô½©ÃÂ£¶¤¼?@!¹«îûúöð±±¾()´`º'#*+.,/\"=:;_ | A-W | Y-Z | a-w | y-z]") == False]
book_ratings = book_ratings[book_ratings['ISBN'].str.contains(r"X{2,}") == False]
book_ratings['ISBN'] = book_ratings['ISBN'].str.replace("-", "")

#### Users

In [ ]:
len(users[users['Location'].str.contains(r"[§Ô½©ÃÂ£¶¤¼@¹«îûúöð±¾]")])/len(users)

0.03256496137819249

In [ ]:
users = users[users['Location'].str.contains(r"[§Ô½©ÃÂ£¶¤¼?@!¹«îûúöð±¾]") == False]

#### GoodBooks
Il dataset `GoodBooks` non presenta problemi di codifica, utilizzando la codifica UTF-8.

Verifichiamo che le colonne `goodreads_book_id` e `best_book_id` non siano uguali (come potrebbe semnbrare vedendo la tabella):

In [ ]:
goodbooks['goodreads_book_id'].equals(goodbooks['best_book_id'])

False

#### GoodBooks-Ratings
Il dataset `GoodBooks-Ratings` non presenta problemi di codifica, utilizzando la codifica UTF-8.

In [ ]:
print('Ecco le dimensioni finali dei dataset:')
print('books --> '+ str(books.shape))
print('book_ratings --> '+ str(book_ratings.shape))
print('goodbooks --> '+ str(goodbooks.shape))
print('goodbooks_ratings --> '+ str(goodbooks_ratings.shape))
print('users --> '+ str(users.shape))

Ecco le dimensioni finali dei dataset:
books --> (264697, 8)
book_ratings --> (1147385, 3)
goodbooks --> (99, 23)
goodbooks_ratings --> (99, 3)
users --> (269710, 3)


# 1. Normalize the location field of Users dataset, splitting into city, region, country.

La varibile `Location` viene splittata in 3 diverse variabili (`city`, `region`, `country`), utilizzando come separatore la virgola.

In [ ]:
split = users['Location'].str.split(', ', 2, expand=True)
split.head()

,0,1,2
0,nyc,new york,usa
1,stockton,california,usa
2,moscow,yukon territory,russia
3,porto,v.n.gaia,portugal
4,farnborough,hants,united kingdom


Una volta create le diverse colonne, queste vengono rinominate per una più facile lettura:

In [ ]:
users_split = users.assign(city=split[0], region=split[1], country=split[2])
users_split.drop('Location', 1, inplace=True) # togliere variabile Location
users_split.head() 

,User-ID,Age,city,region,country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom


# 2. For each book in the Books dataset, compute its average rating.

Dal dataset `book_ratings` è stata calcolata la media degli average rating raggruppando i record in base al loro ISBN. 
Successivamente è stata fatta una left-join tra i due dataset books, così da ottenere per ogni libro la relativa media.

In [ ]:
books_ratings_avg = pd.merge(books[['ISBN', 'Book-Title', 'Book-Author', 'Publisher']], book_ratings.groupby('ISBN')['Book-Rating'].mean(), on='ISBN', how='left').rename(columns={'Book-Rating': 'avg_Book-Rating'})

In [ ]:
books_ratings_avg.head()

,ISBN,Book-Title,Book-Author,Publisher,avg_Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,HarperPerennial,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,0.000000


# 3. For each book in the GoodBooks dataset, compute its average rating.


Viene calcolato il rating sulla base dei valori presenti nel dataset `GoodBooks-Ratings` (`user_id`, `book_id`, `rating`]) considerando come valore di identificazione dei libri il campo `book_id` . Successivamente si esegue una left join per rispondere al quesiti.

In [ ]:
goodbooks_ratings_avg = goodbooks_ratings.groupby(['book_id'], as_index=False).mean()[['book_id','rating']]
goodbooks_ratings_avg.head()

,book_id,rating
0,2,5.0
1,5,4.0
2,8,4.0
3,13,4.0
4,14,5.0


In [ ]:
goodbooks_ratings_avg2 = pd.merge(goodbooks, goodbooks_ratings_avg, on='book_id', how='left')
goodbooks_ratings_avg2.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,rating
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,NaN
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5.0
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,NaN
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,NaN
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,4.0


# 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset merged books. The books that have not been merged together will not appear in merged books

Si crea un nuovo dataset `merged_books` contenente la variabile `Counts` che rappresenta il numero di edizioni per ogni libro.

In [ ]:
merged_books = books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).count()[['Book-Title', 'Book-Author', 'Publisher', 'ISBN']].rename(columns={'ISBN' : 'Counts'})
merged_books.head() 

,Book-Title,Book-Author,Publisher,Counts
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,1
1,Always Have Popsicles,Rebecca Harvin,Rebecca L. Harvin,1
2,Apple Magic (The Collector's series),Martina Boudreau,Amer Cooking Guild,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,Zonderkidz,1
4,Beyond IBM: Leadership Marketing and Finance ...,Lou Mobley,"Teleonet, Incorporated",1


Si decide di tenere solo i record con almeno due pubblicazioni, ovvero con `Counts` > 1.

In [ ]:
merged_books = merged_books[merged_books['Counts'] > 1]
merged_books.head()

,Book-Title,Book-Author,Publisher,Counts
57,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
247,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
455,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
697,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2
709,158 POUND MARRIAGE,John Irving,Pocket,2


# 5. For each book in merged books compute its average rating
The average is computed considering all books in books that have been merged.

Si crea il dataset `merged_books_avg` in cui ogni libro avrà tante righe quante sono le sue pubblicazioni (ciascuna caratterizzata da un relativo rating).
 
Il rating medio finale sarà ottenuto dalla media di questi ratings.

In [ ]:
merged_books_avg = merged_books.merge(books_ratings_avg, on=['Book-Title', 'Book-Author', 'Publisher'])
merged_books_avg.head()

,Book-Title,Book-Author,Publisher,Counts,ISBN,avg_Book-Rating
0,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,0.000000
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792277295,3.400000
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,1566196256,7.250000
3,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,1566196264,6.666667
4,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3,0809288311,5.000000


In [ ]:
merged_books_avg_rtng = merged_books_avg.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).mean().rename(columns={'avg_Book-Rating': 'AVG_Book-Rating'})

In [ ]:
merged_books_avg_rtng.head()

,Book-Title,Book-Author,Publisher,Counts,AVG_Book-Rating
0,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,1.700000
1,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,6.958333
2,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3,3.111111
3,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2,0.000000
4,158 POUND MARRIAGE,John Irving,Pocket,2,2.416667


# 6. For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.

Viene utilizzato lo stesso principio di prima, ma invece che calcolare la media, si ottengono due colonne, riportanti i valori massimo e minimo.

In [ ]:
merged_books_avg['max'] = merged_books_avg.groupby(['Book-Title','Book-Author'], as_index=False)['avg_Book-Rating'].transform('max')
merged_books_avg['min'] = merged_books_avg.groupby(['Book-Title','Book-Author'], as_index=False)['avg_Book-Rating'].transform('min')

Si eliminano le colonne `avg_Book-Rating`, `ISBN` e i record ripetuti.

In [ ]:
merged_books_min_max = merged_books_avg[merged_books_avg.columns.drop(['avg_Book-Rating', 'ISBN'])]
merged_books_min_max = merged_books_min_max.drop_duplicates() # eliminiamo i duplicati
merged_books_min_max.head()

,Book-Title,Book-Author,Publisher,Counts,max,min
0,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,3.40,0.000000
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,7.25,6.666667
4,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3,5.00,0.000000
7,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2,0.00,0.000000
9,158 POUND MARRIAGE,John Irving,Pocket,2,3.50,1.333333


# 7. For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text.

Si crea un nuovo dataset `goodbooks_new`, uguale al dataset `goodbooks`, a cui aggiungiamo la colonna `numero_autori` che conta il **numero** di autori per ciascun libro.

In [ ]:
goodbooks_new = goodbooks
goodbooks_new['numero_autori'] = goodbooks_new['authors'].str.split(',').agg([len])

Successiamente si crea un'altra colonna `author` che contiene il **nome** di ciascun autore per ogni libro. In questo modo, per ogni libro, ci saranno tante righe quanti sono il numero di autori.

In [ ]:
goodbooks_new['author'] = goodbooks_new['authors'].str.split(',')

In [ ]:
goodbooks_new = goodbooks_new.explode('author')

Dato che nella colonna `author` alcune celle contengono uno spazio all'inizio della riga, questo viene rimosso:

In [ ]:
goodbooks_new['author'] = goodbooks_new['author'].str.strip() 

Viene cancellata la colonna `authors` originaria:

In [ ]:
goodbooks_new = goodbooks_new.drop('authors', axis=1)

Infine si crea la colonna `shared_number_of_reviews_with_a_text` data dal rapporto tra il numero di recensioni di ogni libro e il numero di autori e se ne calcola la somma per ciascun autore.

In [ ]:
goodbooks_new['shared_number_of_reviews_with_a_text'] = goodbooks_new['work_text_reviews_count'] / goodbooks_new['numero_autori']

In [ ]:
rev_tot_author = goodbooks_new.groupby('author', as_index=False)['shared_number_of_reviews_with_a_text'].sum()

In [ ]:
rev_tot_author.head()

,author,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.500000
1,Aldous Huxley,20095.000000
2,Alice Sebold,36642.000000
3,Anne Frank,6941.666667
4,Antoine de Saint-Exupéry,6134.250000


# 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

Si crea una nuova tabella `goodbooks_year` che contiene il numero di `shared_number_of_reviews_with_a_text` totali annue per ogni autore.

In [ ]:
goodbooks_year = goodbooks_new.groupby(['original_publication_year', 'author'], as_index=False)['original_publication_year', 'author', 'shared_number_of_reviews_with_a_text'].sum()
goodbooks_year

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,original_publication_year,author,shared_number_of_reviews_with_a_text
0,-720.0,Bernard Knox,1620.2
1,-720.0,E.V. Rieu,1620.2
2,-720.0,Frédéric Mugler,1620.2
3,-720.0,Homer,1620.2
4,-720.0,Robert Fagles,1620.2
...,...,...,...
133,2012.0,E.L. James,25287.0
134,2012.0,Gillian Flynn,121614.0
135,2012.0,John Green,140739.0
136,2012.0,Veronica Roth,55873.0


Si seleziona, in seguito, l'autore o gli autori con il numero massimo di `shared_number_of_reviews_with_a_text` per anno:

In [ ]:
max_year = goodbooks_year[goodbooks_year.groupby('original_publication_year')['shared_number_of_reviews_with_a_text'].transform('max') == goodbooks_year['shared_number_of_reviews_with_a_text']]
max_year

,original_publication_year,author,shared_number_of_reviews_with_a_text
0,-720.0,Bernard Knox,1620.2
1,-720.0,E.V. Rieu,1620.2
2,-720.0,Frédéric Mugler,1620.2
3,-720.0,Homer,1620.2
4,-720.0,Robert Fagles,1620.2
...,...,...,...
129,2009.0,Suzanne Collins,88538.0
130,2010.0,Suzanne Collins,96274.0
131,2011.0,E.L. James,103489.0
135,2012.0,John Green,140739.0


# 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

Viene creata la tabella `diff_ratings` dall'unione tra `goodbooks` e `books_rating_avg` e successivamente si crea in questo nuovo dataset la colonna `difference`, ovvero la differenza tra rating in valore assoluto:

In [ ]:
diff_ratings = goodbooks.merge(books_ratings_avg, left_on='isbn', right_on='ISBN')[['ISBN','average_rating','avg_Book-Rating']]
diff_ratings['difference'] = abs(diff_ratings['average_rating'] - diff_ratings['avg_Book-Rating']) # differenza tra i rating in valore assoluto
diff_ratings

# 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

Viene creato il dizionario `dataset_age` dove vengono inseriti, inizialmente, gli utenti con età sconosciuta o compresa tra 0 e 14 anni:

In [ ]:
dataset_ages = {
    "unknown": users[users.Age.isnull()].copy(),
    "age_0-14": users[(users.Age >= 0) & (users.Age < 15)].copy()
}

Si crea una lista di range d'età, da 15 al valore massimo della variabile `Age` (244), e si costruisce l'intero dizionario.

In [ ]:
ages = list(range(15, int(users.Age.max()), 10))
for i in range(len(ages)):
    try:
        age_range = f"age_{ages[i]}-{ages[i+1]-1}"
        tmp = users[(users.Age >= ages[i]) & (users.Age < ages[i+1])].copy()
    except IndexError:
        age_range = f"age_>={ages[i]}"
        tmp = users[(users.Age >= ages[i])].copy()
    dataset_ages[age_range] = tmp

Ecco il risultato:

In [ ]:
for k in list(dataset_ages.keys()):
    print(f"Age range: {k}")
    print(f"Minimum Age: {dataset_ages[k].Age.min()}")
    print(f"Maximum Age: {dataset_ages[k].Age.max()}")
    print("\n")

# 11. Find the books that appear only in the goodbooks datasets.

Si trovano i libri contenuti sia nel dataset `books` che nel dataset `gb_books`:

In [ ]:
gb_books = goodbooks.rename(columns={'isbn':'ISBN'})

In [ ]:
good_in_books = gb_books.merge(books, how='inner', on=['ISBN'])

In [ ]:
len(good_in_books) 

In [ ]:
good_in_books

Si costruisce la tabella `goodbooks_only` che esclude da `gb_books` gli elementi di `good_in_books`:

In [ ]:
goodbooks_only = gb_books.loc[~gb_books['ISBN'].isin(good_in_books['ISBN'])].copy()
len(goodbooks_only)

# 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

Si contano le volte che ciascun libro appare nel dataset `books` e si ordina il nuovo dataset (`n_times_book`) in modo decrescente:

In [ ]:
n_times_book = books.groupby(['Book-Title', 'Book-Author'], as_index=False).count()[['Book-Title', 'Book-Author', 'ISBN']].rename(columns={'ISBN' : 'Counts'})
n_times_book.sort_values(by=['Counts'], inplace=True, ascending=False)
n_times_book.head()

Infine si seleziona il libro che è apparso il maggior numero di volte:

In [ ]:
n_times_book[n_times_book['Counts'] == n_times_book['Counts'].max()]

# 13. Find the author with the highest average rating according to the goodbooks datasets.

Per ogni autore si calcola la media delle sue recensioni e si ordina la nuova tabella (`author_avg`) in modo decrescente:

In [ ]:
authors_avg = goodbooks_new.groupby(['author'], as_index=False).mean()[['author','average_rating']]
authors_avg.sort_values(by=['average_rating'], inplace=True, ascending=False)
authors_avg.head()

Infine si seleziona l'autore con la media di recensioni più alta:

In [ ]:
authors_avg[authors_avg['average_rating'] == authors_avg['average_rating'].max()]